In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
import time
import gc
import os
import random
from PIL import Image
from sklearn.metrics import f1_score, accuracy_score

import timm
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models as models

device = torch.device('cuda')

In [3]:
def img_load(path):
    #BGR이미지를 RGB로 변경 (모든 행과 열의 요소를 역순으로 뒤집음)
    img = Image.open(path)
    img = img.convert('RGB')
    return img

def score_function(real, pred):
    score = f1_score(real, pred, average='macro')
    return score

In [4]:
t_train_rotate = T.Compose([
                    T.RandomAffine(360), # 랜덤 회전
                    T.ToTensor(),
                    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

t_train_vflip = T.Compose([
                    T.RandomVerticalFlip(1.0), # 랜덤 상하 반전
                    T.ToTensor(),
                    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

t_train_hflip = T.Compose([
                    T.RandomHorizontalFlip(1.0), # 랜덤 좌우 반전
                    T.ToTensor(),
                    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

t_test = T.Compose([
                    T.ToTensor(),
                    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        if self.transform == 'badaug':
            if 'good' not in self.labels:
                tmpimg = self.img_paths[idx]
                label = self.labels[idx]
                img = self.transform(tmpimg)
        elif self.transform != 'badaug':
            tmpimg = self.img_paths[idx]
            label = self.labels[idx]
            img = self.transform(tmpimg)
        return img, label

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b2', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
train_png = sorted(glob('/content/gdrive/My Drive/data/open/train256/*.png'))
test_png = sorted(glob('/content/gdrive/My Drive/data/open/test256/*.png'))

In [8]:
train_y = pd.read_csv('/content/gdrive/My Drive/data/open/train_df.csv')

train_labels = train_y['label']

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [9]:
train_imgs = [img_load(m) for m in tqdm(train_png)]

  0%|          | 0/4277 [00:00<?, ?it/s]

In [10]:
os.cpu_count()

2

In [11]:
batch_size = 16
epochs = 40

train_dataset_default = Custom_dataset(np.array(train_imgs), np.array(train_labels), transform=t_test)
train_dataset_rotate = Custom_dataset(np.array(train_imgs), np.array(train_labels), transform=t_train_rotate)
train_dataset_vflip = Custom_dataset(np.array(train_imgs), np.array(train_labels), transform=t_train_vflip)
train_dataset_hflip = Custom_dataset(np.array(train_imgs), np.array(train_labels), transform=t_train_hflip)
train_dataset = torch.utils.data.ConcatDataset([train_dataset_default, train_dataset_rotate, train_dataset_vflip, train_dataset_hflip])

In [12]:
del train_imgs, train_dataset_default, train_dataset_rotate, train_dataset_vflip, train_dataset_hflip, train_labels

In [13]:
gc.collect()

389

In [14]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=2)

In [15]:
del train_dataset

In [16]:
gc.collect()

0

In [17]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00015)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b2_ra-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_ra-bcdf34b7.pth


epoch : 1/40    time : 167s/6501s
TRAIN    loss : 0.51052    f1 : 0.51352
epoch : 2/40    time : 160s/6087s
TRAIN    loss : 0.14256    f1 : 0.82681
epoch : 3/40    time : 160s/5912s
TRAIN    loss : 0.07745    f1 : 0.90944
epoch : 4/40    time : 159s/5733s
TRAIN    loss : 0.05511    f1 : 0.94522
epoch : 5/40    time : 158s/5547s
TRAIN    loss : 0.05175    f1 : 0.94589
epoch : 6/40    time : 160s/5446s
TRAIN    loss : 0.03736    f1 : 0.95919
epoch : 7/40    time : 159s/5233s
TRAIN    loss : 0.04075    f1 : 0.95940
epoch : 8/40    time : 159s/5096s
TRAIN    loss : 0.03445    f1 : 0.96346
epoch : 9/40    time : 158s/4892s
TRAIN    loss : 0.02986    f1 : 0.97152
epoch : 10/40    time : 158s/4731s
TRAIN    loss : 0.02805    f1 : 0.97587
epoch : 11/40    time : 159s/4606s
TRAIN    loss : 0.02779    f1 : 0.97474
epoch : 12/40    time : 160s/4467s
TRAIN    loss : 0.02331    f1 : 0.97767
epoch : 13/40    time : 159s/4297s
TRAIN    loss : 0.02171    f1 : 0.97816
epoch : 14/40    time : 157s/4086s

KeyboardInterrupt: ignored

In [18]:
del train_loader
gc.collect()

8

In [19]:
torch.cuda.empty_cache()
gc.collect()

0

In [20]:
test_imgs = [img_load(n) for n in tqdm(test_png)]

  0%|          | 0/2154 [00:00<?, ?it/s]

In [21]:
test_dataset = Custom_dataset(np.array(test_imgs), np.array(['tmp']*len(test_imgs)), transform=t_test)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=2)

In [22]:
del test_imgs, test_dataset
gc.collect()

358

In [23]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [24]:
del test_loader
gc.collect()

0

In [25]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [26]:
submission = pd.read_csv('/content/gdrive/My Drive/data/open/sample_submission.csv')

submission['label'] = f_result

submission
#2154 rows x 2 cols

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [27]:
submission.to_csv('/content/gdrive/My Drive/workspace/mysubmission_01.csv', index=False)